In [ ]:
## Document loaders with langchain
#https://python.langchain.com/docs/modules/data_connection/document_loaders/json

#! pip install jq

In [ ]:
### The course will show the pip installs you would need to install packages on your own machine.
#! pip install pypdf 

In [ ]:
from langchain.document_loaders import PyPDFLoader
import os

In [ ]:
span_doc = os.listdir('./spa-assistant-agent/spa-data/')

In [ ]:
span_doc

In [ ]:
loaders = []
for n in span_doc:
    file = "./spa-assistant-agent/spa-data/" + n
    loader = PyPDFLoader(file)
    pages = loader.load()
    loaders.append(loader.load())

## Embeddings
https://github.com/aws-samples/rag-using-langchain-amazon-bedrock-and-opensearch/blob/main/ask-bedrock-with-rag.py

In [ ]:
import boto3
from langchain.embeddings import BedrockEmbeddings

bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

bedrock_embedding_model_id = 'amazon.titan-embed-text-v1'

In [ ]:
def create_langchain_vector_embedding_using_bedrock(bedrock_client, bedrock_embedding_model_id):
    bedrock_embeddings_client = BedrockEmbeddings(
        client=bedrock_client,
        model_id=bedrock_embedding_model_id)
    return bedrock_embeddings_client

In [ ]:
bedrock_embeddings_client = create_langchain_vector_embedding_using_bedrock(bedrock_client, bedrock_embedding_model_id)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 150
)

In [ ]:
splits = []
for doc in loaders:
    splits.append(text_splitter.split_documents(doc))

## Vectorstores

https://python.langchain.com/docs/integrations/vectorstores/chroma

! pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = './spa-assistant-agent/vectordb/docs/chroma/'

In [ ]:
for split in splits :
    vectordb = Chroma.from_documents(
        documents=split,
        embedding=bedrock_embeddings_client,
        persist_directory=persist_directory
    )
   #print(vectordb)

### TEST the vectordb

In [ ]:
query = "Which massage is the best for relieving headache?"
vector_result = vectordb.similarity_search(query,k=2)
vector_result